In [1]:
FILE_PATH = '../../data/raw/amz_products_small.jsonl.gz'


In [2]:
%%time
import extract
import fire



def run():
    data = extract.from_file(FILE_PATH)
    # print(data.head(10))
    
run()
    
    
    
# if __name__ == "__main__":
#     fire.Fire(run)

CPU times: user 1min, sys: 3.81 s, total: 1min 4s
Wall time: 1min 3s


In [4]:
%%time
# Combine text columns
data['text'] = data['title'] + " " + data['description'] + " " + data['brand'] + " " + data['feature'].apply(' '.join) + " " + data['category']

NameError: name 'data' is not defined

In [2]:
%%time
import pandas as pd

# Load JSON data
df = pd.read_json(FILE_PATH, lines=True)
df.head(5)



: 

In [3]:
%%time
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws, col

spark = SparkSession.builder \
    .appName("BigDataProcessing") \
    .master("local[*]") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

# Load JSON data
df = spark.read.json(FILE_PATH)


CPU times: user 5.2 ms, sys: 4.6 ms, total: 9.8 ms
Wall time: 22.1 s


In [4]:
%%time
from pyspark.sql.functions import concat_ws, col

# Combine text columns
df = df.withColumn("text", concat_ws(" ", col("title"), col("description"), col("brand"), col("feature"), col("category")))

# Trigger an action to evaluate the transformation
df.show(5)

# Stop Spark session to free up resources
spark.stop()


+--------+--------------------+----------+---------------+--------------------+--------------------+--------------------+--------------------+----------+-------+--------------------+--------------------+
|also_buy|           also_view|      asin|          brand|            category|         description|             feature|               image|  main_cat|  price|               title|                text|
+--------+--------------------+----------+---------------+--------------------+--------------------+--------------------+--------------------+----------+-------+--------------------+--------------------+
|      []|                  []|B00ADZ3WUM|            NSI|[Automotive, Exte...|[Bumpersticker: A...|[Official License...|                  []|Automotive|  $4.68|NSI - A Day Witho...|NSI - A Day Witho...|
|      []|                  []|B005VII5IU| General Motors|[Automotive, Repl...|[This is the offi...|[This is the offi...|                  []|Automotive|$213.16|Genuine GM Parts ...|Ge